### Required Libaries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import sklearn
from sklearn.decomposition import PCA # Principal Component analysis
from sklearn.preprocessing import StandardScaler # Scaling purpose
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor # RandomForestRegressor
# SVR
from sklearn import svm
from sklearn.svm import SVR
# PLSR
from sklearn.cross_decomposition import PLSRegression
# MLR
from sklearn.linear_model import LinearRegression
#ensemble --xgboost
from sklearn import ensemble
# adaboost
from sklearn.ensemble import AdaBoostRegressor
# Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
# Multi layer Perceptron
from sklearn.neural_network import MLPRegressor
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
# Save the model using pickle
import pickle
# Save the modle using joblib
import joblib
#performance metrics
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

## **Dataset**

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/FYP/code/terra_dataset_kohat.csv")

In [ ]:
# Five 5 rows
dataset.head()

,Latitude,Longitude,b01,b02,b03,b04,b05,b06,b07,NDVI,...,BI,Clay,Silt,Sand,pH,CaCo3,Organic_Matter,Nitrogen,Texture Class,EC
0,33.601632,71.265752,0.0853,0.2497,0.0453,0.0796,0.2749,0.2102,0.1358,0.490746,...,0.263868,7,70,24,7.20,28,0.86,0.04,Silt Loam,0.315
1,33.605954,71.253803,0.1253,0.2374,0.0698,0.1077,0.2706,0.2405,0.1846,0.309071,...,0.268438,9,64,28,7.25,12,0.72,0.04,Silt Loam,0.258
2,33.605399,71.235295,0.1504,0.2547,0.0746,0.1161,0.2983,0.2841,0.2229,0.257467,...,0.295791,7,74,20,7.30,14,0.69,0.03,Silt Loam,0.279
3,33.580629,71.209936,0.1100,0.2570,0.0535,0.0901,0.2911,0.2594,0.1699,0.400545,...,0.279551,13,62,26,7.42,10,1.52,0.08,Silt Loam,0.279
4,33.576281,71.192251,0.1186,0.2236,0.0629,0.1018,0.2555,0.2345,0.1739,0.306838,...,0.253107,11,58,32,7.49,10,1.73,0.09,Silt Loam,0.323


In [ ]:
# dataset.info()

In [ ]:
# dataset.describe()

In [ ]:
# Check null/empty/0 values
# dataset.isnull().sum() 

## **Bands Correlations with 'EC'**

In [8]:
bands = ['b01','b02','b03','b04','b05','b06','b07','EC']
bands_corr =  dataset.loc[:,bands]
corr1 = bands_corr.corr(method = "pearson")
print(corr1)

          b01       b02       b03       b04       b05       b06       b07  \
b01  1.000000  0.223332  0.925227  0.959333  0.392922  0.818049  0.936327   
b02  0.223332  1.000000  0.280283  0.303294  0.723716  0.342278  0.180884   
b03  0.925227  0.280283  1.000000  0.967229  0.337289  0.693131  0.805600   
b04  0.959333  0.303294  0.967229  1.000000  0.335473  0.707988  0.852755   
b05  0.392922  0.723716  0.337289  0.335473  1.000000  0.691440  0.473564   
b06  0.818049  0.342278  0.693131  0.707988  0.691440  1.000000  0.907978   
b07  0.936327  0.180884  0.805600  0.852755  0.473564  0.907978  1.000000   
EC  -0.120900  0.166062 -0.083170 -0.003805 -0.181765 -0.338927 -0.248779   

           EC  
b01 -0.120900  
b02  0.166062  
b03 -0.083170  
b04 -0.003805  
b05 -0.181765  
b06 -0.338927  
b07 -0.248779  
EC   1.000000  


## **Vegetation Indices Correlations with 'EC'**

In [9]:
vegetation = ['NDVI','DVI','EVI','SAVI','OSAVI','EVI2','MASAVI2','BI','EC']
vegetation_corr = dataset.loc[:,vegetation]
corr2 = vegetation_corr .corr(method = "pearson")
print(corr2)

             NDVI       DVI       EVI      SAVI     OSAVI      EVI2   MASAVI2  \
NDVI     1.000000  0.944019  0.974612  0.983846  0.995936  0.981459  0.976352   
DVI      0.944019  1.000000  0.987775  0.987781  0.969834  0.989615  0.992763   
EVI      0.974612  0.987775  1.000000  0.995584  0.989100  0.995782  0.995750   
SAVI     0.983846  0.987781  0.995584  1.000000  0.995966  0.999854  0.999135   
OSAVI    0.995936  0.969834  0.989100  0.995966  1.000000  0.994691  0.991764   
EVI2     0.981459  0.989615  0.995782  0.999854  0.994691  1.000000  0.999670   
MASAVI2  0.976352  0.992763  0.995750  0.999135  0.991764  0.999670  1.000000   
BI      -0.293344  0.031815 -0.094633 -0.121956 -0.208657 -0.108145 -0.082860   
EC       0.204823  0.222441  0.220543  0.214531  0.209692  0.220235  0.224614   

               BI        EC  
NDVI    -0.293344  0.204823  
DVI      0.031815  0.222441  
EVI     -0.094633  0.220543  
SAVI    -0.121956  0.214531  
OSAVI   -0.208657  0.209692  
EVI2    -

## **Salinity Indices Correlations with 'EC'**

In [10]:
salinity = ['NDSI','VSSI','SI','SI1','SI2','SI3','SI4','SI5','EC']
salinity_corr = dataset.loc[:,salinity ]
corr3 = salinity_corr.corr(method = "pearson")
print(corr3)

          NDSI      VSSI        SI       SI1       SI2       SI3       SI4  \
NDSI  1.000000 -0.472037  0.878894  0.893906  0.762649  0.437904  0.898660   
VSSI -0.472037  1.000000 -0.750682 -0.789810 -0.928412 -0.989471 -0.789656   
SI    0.878894 -0.750682  1.000000  0.945270  0.912004  0.724695  0.949790   
SI1   0.893906 -0.789810  0.945270  1.000000  0.957584  0.784668  0.999721   
SI2   0.762649 -0.928412  0.912004  0.957584  1.000000  0.911756  0.958366   
SI3   0.437904 -0.989471  0.724695  0.784668  0.911756  1.000000  0.781475   
SI4   0.898660 -0.789656  0.949790  0.999721  0.958366  0.781475  1.000000   
SI5  -0.288458  0.095280 -0.434743 -0.134277 -0.154195  0.001788 -0.152626   
EC   -0.204823 -0.006517 -0.052433 -0.070201 -0.074005  0.074027 -0.076122   

           SI5        EC  
NDSI -0.288458 -0.204823  
VSSI  0.095280 -0.006517  
SI   -0.434743 -0.052433  
SI1  -0.134277 -0.070201  
SI2  -0.154195 -0.074005  
SI3   0.001788  0.074027  
SI4  -0.152626 -0.076122  
SI5

## **Features and Labels**

In [11]:
features_columns = ['b01','b02','b03','b04','b05','b06','b07','NDVI','DVI','SAVI','OSAVI','EVI2','MASAVI2'
                    ,'NDSI','VSSI','SI','SI1','SI2','SI3','SI4','SI5']
# features_columns = ['b02','NDVI','DVI','EVI','SAVI','OSAVI','EVI2','MASAVI2','SI3','SI5'] #based on positive correlation

In [12]:
features =dataset.loc[:,features_columns]

In [13]:
# features.info()


***Noise Removal***

In [14]:
from scipy.signal import savgol_filter
feature_filtered = savgol_filter(features,21,3) #window = ant ositive odd number, 3rd parameter= polyorder

In [15]:
feature_filtered

array([[-0.08309124,  0.10026449,  0.22957472, ...,  0.09027513,
         0.26492278,  0.49894975],
       [-0.01264015,  0.13598739,  0.236255  , ...,  0.1211688 ,
         0.31011433,  0.55785249],
       [ 0.01992991,  0.16105175,  0.25431687, ...,  0.1135837 ,
         0.30744818,  0.56100461],
       ...,
       [-0.00395927,  0.14830887,  0.25084103, ...,  0.10936309,
         0.30052746,  0.55185513],
       [-0.08309124,  0.10026449,  0.22957472, ...,  0.09027513,
         0.26492278,  0.49894975],
       [-0.02703803,  0.1334592 ,  0.24286831, ...,  0.11980471,
         0.31216107,  0.56536404]])

***PCA Implementation***

In [16]:
pca1 = PCA(n_components = 7)
pca_first = pca1.fit_transform(features)

In [17]:
print(pca_first.shape)

(50, 7)


In [18]:
pca_first

array([[ 3.44777731e-01, -4.80150877e-02,  1.71852427e-02,
        -1.30479553e-02, -6.21958240e-03, -6.88900870e-03,
         1.99433554e-03],
       [-6.41794914e-02,  6.14578244e-02, -3.96472596e-02,
        -1.20919845e-02,  6.03385041e-03, -4.27639838e-03,
         2.55102687e-03],
       [-2.65700897e-01, -7.09422314e-02,  1.71471573e-02,
        -2.15441211e-02,  1.69026562e-02,  2.79606147e-03,
         3.45203138e-03],
       [ 9.99279325e-02, -9.16566307e-02,  5.16959893e-02,
        -1.23262672e-02,  1.01234966e-02,  8.91305674e-03,
        -1.45237663e-03],
       [-3.31621323e-02,  1.55415708e-01, -1.40062689e-02,
        -1.67448801e-02,  5.47068287e-03, -1.56504021e-03,
        -3.68213686e-03],
       [ 2.42031326e-02,  7.09912222e-02, -5.85932620e-03,
        -6.03079324e-03,  6.31317555e-03,  5.37324062e-03,
        -6.64413537e-05],
       [ 3.66224830e-02, -8.53808562e-02,  2.74626283e-02,
         2.42967118e-03,  1.24171077e-03, -2.26493567e-03,
        -3.2166848

***Scaling***

In [19]:
scaler = StandardScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(pca_first))

In [20]:
print(features_scaled)

           0         1         2         3         4         5         6
0   2.109464 -0.519323  0.576703 -0.808119 -0.803252 -1.176223  0.484486
1  -0.392671  0.664718 -1.330484 -0.748911  0.779265 -0.730148  0.619723
2  -1.625646 -0.767300  0.575425 -1.334325  2.182959  0.477397  0.838605
3   0.611392 -0.991343  1.734816 -0.763422  1.307438  1.521806 -0.352827
4  -0.202897  1.680951 -0.470023 -1.037086  0.706533 -0.267213 -0.894505
5   0.148083  0.767829 -0.196627 -0.373514  0.815340  0.917422 -0.016141
6   0.224068 -0.923465  0.921592  0.150481  0.160366 -0.386713 -0.781432
7   0.801543  0.198690  0.305541  0.602942  0.169398  1.832224 -0.862782
8   0.437045  0.398407 -0.495257  0.611557  1.190236  0.434182  0.173129
9   0.116607  0.187664 -0.164484  0.087469  0.926520 -0.335457 -0.830656
10  0.953098 -0.387245  0.026420 -0.409582 -0.977899  1.398471  1.537611
11 -0.740741  1.885537 -0.550357 -0.567369 -0.846653 -0.180909 -0.050446
12 -1.561925 -0.233782  0.803227 -0.080339 -1.04563

In [21]:
label_columns = ['EC']
labels = dataset.loc[:,label_columns]

In [22]:
print(pca_first.shape)
print(labels.shape)

(50, 7)
(50, 1)


## **Train Test Splitting**

In [23]:
x_train,x_test,y_train,y_test = train_test_split(pca_first ,labels,test_size = 0.2,shuffle = True)

In [24]:
print("x_train : ",x_train.shape)
print("x_test : ",x_test.shape)
print("y_train : ",y_train.shape)
print("y_test : ",y_test.shape)

x_train :  (40, 7)
x_test :  (10, 7)
y_train :  (40, 1)
y_test :  (10, 1)


## **Models**

###  **1. Random Forest Regressor**

In [25]:
rf = RandomForestRegressor(n_estimators =35,max_depth=2, random_state=0)

In [26]:
model1 = rf.fit(x_train,y_train.values.ravel()) #training

In [27]:
y_pred1 = model1.predict(x_test)  #predicting

In [28]:
y_test,y_pred1

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186, array([0.24339268, 0.25353128, 0.20267435, 0.23087656, 0.19565212,
        0.20341009, 0.24475722, 0.28136122, 0.21109308, 0.21595207]))

## **Performance Metrics**
### **R2, RMSE, MAE, MSE**

In [29]:
r2 = r2_score(y_test,y_pred1)  #R2
mae= mean_absolute_error(y_test,y_pred1) #mae
rmse = mean_squared_error(y_test,y_pred1) #rmse
mse = mean_squared_error(y_test,y_pred1, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -0.9382355934441644
RMSE:  0.003123582952970879
MSE:  0.05588902354640739
MAE:  0.04206890804249943


###  **2.SVR**

In [30]:
svr = SVR(kernel='rbf', C=32, gamma=0.0313) #kernel = radial basis function

In [31]:
model2 =svr.fit(x_train,y_train.values.ravel()) #fitting

In [32]:
y_pred2 = model2.predict(x_test) #predicting

In [33]:
# y_test,y_pred2

In [34]:
r2 = r2_score(y_test,y_pred2)  #R2
mae= mean_absolute_error(y_test,y_pred2) #mae
rmse = mean_squared_error(y_test,y_pred2) #rmse
mse = mean_squared_error(y_test,y_pred2, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -1.3013945082559841
RMSE:  0.0037088353337250157
MSE:  0.06090020799410307
MAE:  0.048797214940615084


### **3. Multiple Linear Regression**

In [35]:
mlr = LinearRegression()

In [36]:
model3 = mlr.fit(x_train,y_train) #fitting

In [37]:
y_pred3= model3.predict(x_test) #predicting

In [38]:
y_test,y_pred3

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186, array([[0.22233191],
        [0.30671875],
        [0.19482678],
        [0.24214671],
        [0.16114736],
        [0.18045769],
        [0.32840772],
        [0.30647587],
        [0.16780248],
        [0.18102229]]))

In [39]:
r2 = r2_score(y_test,y_pred3)  #R2
mae= mean_absolute_error(y_test,y_pred3) #mae
rmse = mean_squared_error(y_test,y_pred3) #rmse
mse = mean_squared_error(y_test,y_pred3, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -2.7963162387852263
RMSE:  0.006117991397776723
MSE:  0.07821759007906548
MAE:  0.05894720792319315


### **4. PLSR**

In [40]:
plsr = PLSRegression(n_components=7)

In [41]:
model4 = plsr.fit(x_train,y_train) #fitting

In [42]:
y_pred4 = model4.predict(x_test) #predicting

In [43]:
y_test,y_pred4

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186, array([[0.22233191],
        [0.30671875],
        [0.19482678],
        [0.24214671],
        [0.16114736],
        [0.18045769],
        [0.32840772],
        [0.30647587],
        [0.16780248],
        [0.18102229]]))

In [44]:
r2 = r2_score(y_test,y_pred4)  #R2
mae= mean_absolute_error(y_test,y_pred4) #mae
rmse = mean_squared_error(y_test,y_pred4) #rmse
mse = mean_squared_error(y_test,y_pred4, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -2.7963162387852223
RMSE:  0.006117991397776716
MSE:  0.07821759007906544
MAE:  0.05894720792319309


### **5.XGBR (Extreme Gradient Boost Regressor)**

In [45]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.001,
    "loss": "squared_error",
}
xgbr = ensemble.GradientBoostingRegressor(**params)
model5  = xgbr.fit(x_train, y_train.values.ravel())  #fitting

In [46]:
y_pred5 = model5.predict(x_test) #predicting

In [47]:
y_test,y_pred5

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186, array([0.22664158, 0.25148804, 0.22079625, 0.2338735 , 0.20998285,
        0.2221695 , 0.23982692, 0.26436591, 0.22385014, 0.24004881]))

In [48]:
r2 = r2_score(y_test,y_pred5)  #R2
mae= mean_absolute_error(y_test,y_pred5) #mae
rmse = mean_squared_error(y_test,y_pred5) #rmse
mse = mean_squared_error(y_test,y_pred5, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -0.8884119656328915
RMSE:  0.003043289187335344
MSE:  0.05516601478569341
MAE:  0.04444208517357371


### **6.AdaBoost**

In [49]:
ada = AdaBoostRegressor(random_state=0, n_estimators=100)
model6 = ada.fit(x_train, y_train.values.ravel()) #fitting

In [50]:
y_pred6 = model6.predict(x_test) #predicting

In [51]:
y_test,y_pred6

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186, array([0.214     , 0.273     , 0.21911765, 0.23065   , 0.181     ,
        0.21207692, 0.22473333, 0.27230769, 0.213     , 0.21207692]))

In [52]:
r2 = r2_score(y_pred6,y_test)  #R2
mae= mean_absolute_error(y_test,y_pred6) #mae
rmse = mean_squared_error(y_test,y_pred6) #rmse
mse = mean_squared_error(y_test,y_pred6, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -3.4977014083546845
RMSE:  0.0032128161170470416
MSE:  0.056681708840216186
MAE:  0.044242722473604834


### **7. Decision Tree Regressor**

In [53]:
decTree= DecisionTreeRegressor(random_state=0)
model7 = decTree.fit(x_train, y_train) #fitting

In [54]:
y_pred7 = model7.predict(x_test) #predicting

In [55]:
y_test,y_pred7

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186,
 array([0.21 , 0.323, 0.181, 0.213, 0.168, 0.182, 0.213, 0.319, 0.182,
        0.162]))

In [56]:
r2 = r2_score(y_test,y_pred7)  #R2
mae= mean_absolute_error(y_test,y_pred7) #mae
rmse = mean_squared_error(y_test,y_pred7) #rmse
mse = mean_squared_error(y_test,y_pred7, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -2.61891583310581
RMSE:  0.0058321000000000015
MSE:  0.07636818709384165
MAE:  0.058300000000000005


### **8.Multi layer perceptron** 

In [57]:
mlp = MLPRegressor(random_state=1, max_iter=5000)
model8 = mlp.fit(x_train, y_train.values.ravel())

In [58]:
y_pred8 = model8.predict(x_test)

In [59]:
y_test,y_pred8

(       EC
 2   0.279
 45  0.207
 16  0.242
 5   0.250
 20  0.203
 30  0.205
 39  0.203
 36  0.141
 18  0.152
 21  0.186, array([0.28021071, 0.20268645, 0.19497233, 0.21442984, 0.23663521,
        0.23212652, 0.24517674, 0.22242757, 0.20330391, 0.20047101]))

In [60]:
r2 = r2_score(y_test,y_pred8)  #R2
mae= mean_absolute_error(y_test,y_pred8) #mae
rmse = mean_squared_error(y_test,y_pred8) #rmse
mse = mean_squared_error(y_test,y_pred8, squared=False) #mse
# print
print("R^2: ",r2)
print("RMSE: ",rmse)
print("MSE: ",mse)
print("MAE: ",mae)

R^2:  -0.03098332980009788
RMSE:  0.0016614914949726466
MSE:  0.040761397117525876
MAE:  0.03382630621211359


### **9. Naive Bayes Regressor**

In [61]:
# nb = GaussianNB()
# model9 = nb.fit(x_train, y_train.values.ravel()) #fitting but give error
# y_pred9 = model9.predict(x_test)

In [62]:
# y_test,y_pred9
# r2 = r2_score(y_test,y_pred9)  #R2
# mae= mean_absolute_error(y_test,y_pred9) #mae
# rmse = mean_squared_error(y_test,y_pred9) #rmse
# mse = mean_squared_error(y_test,y_pred9, squared=False) #mse
# # print
# print("R^2: ",r2)
# print("RMSE: ",rmse)
# print("MSE: ",mse)
# print("MAE: ",mae)

### **Save the model**

In [63]:
# 1-Method:  Using Pickle
# file_name = "file_name.pickle"

# pickle.dump(model_name, open(file_name, "wb")) # save model
# loaded_model = pickle.load(open(file_name, "rb")) # load model


# prediction = loaded_model.predict(give_value)# use loaded model to compute predictions

# 2-Method: Using joblib
# file_name = "file_name.joblib"

# joblib.dump(rf, file_name) # save model

# loaded_model = joblib.load(file_name) # load model

# prediction = loaded_model.predict(give_value) # use loaded model to compute predictions
